# HSMA 3B: Exercise 2 - Importing Files and Basic Plots with Geopandas

Import the required packages.

## 1. Importing a geojson

Import the stats19 dataframe from before (stats_19_counts_by_msoa_normalised_3857.geojson)

Plot the dataframe.

## 2. Creating a geopandas dataframe from a csv

Import the dataframe of incidents with coordinates (stats19_collisions_point_data_2022.csv) using pandas.

Turn this into a geodataframe, using ESPG:3857 as the coordinate reference system.

Plot the points, colouring them by incident type.

## 3. Joining a geopandas dataframe to a pandas dataframe

Load in the dataframe of clients in Oxford (oxford_smoker_df.csv) who have reported that they are smokers using Pandas. 

(don't worry - this is a fake dataset!)

In [ ]:
smokers_df = ...

Let's take a look at the first 5 rows of this dataframe.

In [ ]:
smokers_df.head()

This dataframe only contains the postcodes of the clients. We want to visualise counts of smokers by LSOA to help us think about where we might place some smoking cessation services - but our database doesn't have a table available to do the postcode-LSOA lookup. 

Instead, we're going to use an application-programming-interface (api) to get this data. We will send it the list of postcodes in the format it expects, and we will then turn the data it sends back into a dataframe and join this up to our smokers_df. 

Just run the code below to get a dataframe containing 

In [ ]:
import requests

# Find the unique postcodes so we only request the 
# data once for each postcode rather than once per
# line of patient data
postcode_list = smokers_df['Postcode'].unique()

# Create an empty list for our results
results = []

# Iterate through the postcodes 100 at a time, returning the data from the API
for i in range(int(np.ceil(len(postcode_list)/100))):

    postcode_list_filtered = postcode_list[i*100:(((i+1)*100)-1)]
    print(f"Collecting postcodes {i*100} to {(((i+1)*100)-1)}")

    # Send API request, passing in your postcodes as a list
    postcode_lookup = requests.post(
        "https://api.postcodes.io/postcodes", 
        json={"postcodes": postcode_list_filtered.tolist()}
    )

    results.append(postcode_lookup)

# Turn the results into a list of pandas dataframes
# (1 dataframe per 100 results)
results_dfs = [pd.json_normalize(i.json()['result'], sep='_') 
for i in results]

# join all of our dataframes into a single dataframe.
postcodes_df = pd.concat(results_dfs)

Take a look at the contents of this dataframe using the 'head' method. Try looking at the first 20 lines instead of the first 5 this time. 

We don't need all of these columns - just one column that we can join to our dataframe on, and another for 

In [ ]:
postcodes_df_subset = ...

Now we can join this to our dataframe using the pandas 'merge' method.

In [ ]:
smokers_df_lsoas = pd.merge(
    left=smokers_df,
    right=postcodes_df_subset,
    left_on=..., 
    right_on=...,
    how=...
)

Now we've got the LSOAs, we can count the number of patients per LSOA. 

In [ ]:
smokers_counts_by_lsoa_df = ...

Now we will load in the LSOA geopackage (Lower_layer_Super_Output_Areas_2021_EW_BGC_V3_2542665517405622314.gpkg). This gives us the boundaries of each LSOA. Load this in using the geopands read_file function. 

In [ ]:
lsoa_boundaries = ...

Now we are going to join our LSOA boundaries to our data files. 

Finally, let's plot the dataframe, colouring by the number of smokers per LSOA.

## Bonus exercises

### 1. Write your smoker df counts to a geojson file

###  2. Join your smoker counts dataframe to a dataframe of LSOA population counts, calculate the number of smokers per 1000 population, and plot the dataframe again

You can download the mid-year estimates from this webpage: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/lowersuperoutputareamidyearpopulationestimates

You will need to use the pandas read_excel method instead. You may wish to look up the arguments for that method on Google. You may need to specify the sheet, skip rows, or select a subset of columns.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5b9a0342-1ee9-4024-886e-fd3cfaca1c00' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>